<a href="https://colab.research.google.com/github/merkarii/SVDAlgorithm-ContentFiltering/blob/main/CollaborativeFiltering_SVD_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195838 sha256=4472c55b3b8f10a7eb84603363206cbac6fbf4be4de2f075cc25b17d069288ce
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [ ]:
!pip install pandas
import pandas as pd
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

import pandas as pd

In [ ]:
# Assuming you have a CSV file with columns: user_id, movie_id, and rating
ratings_data = pd.read_csv("/content/drive/MyDrive/ratings.csv")


In [ ]:
# Prepare the data for scikit-surprise
reader = Reader(rating_scale=(1, 5)) 
# Assuming the rating scale is between 1 and 5
data = Dataset.load_from_df(ratings_data[['userId', 'movieId', 'rating']], reader)

In [ ]:
# Split the data into a training set and a test set
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Train the SVD algorithm on the training set
svd = SVD()
svd.fit(trainset)

In [ ]:
# Make predictions on the test set and evaluate the performance
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8781


0.8781246522508156

In [ ]:
# Function to predict ratings for a user and movie
def predict_rating(userId, movieId):
    return svd.predict(userId, movieId).est

In [ ]:
# Function to recommend movies for a user
def recommend_movies(userId, n_recommendations=5):
    # Get the unique movie ids
    movie_ids = ratings_data['movieId'].unique()
    
    # Predict ratings for all movies and sort by predicted rating
    predicted_ratings = [(movieId, predict_rating(userId, movieId)) for movieId in movie_ids]
    sorted_predicted_ratings = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)
    
    # Filter out the movies the user has already rated
    rated_movies = ratings_data[ratings_data['userId'] == userId]['movieId']
    recommendations = [movieId for movieId, _ in sorted_predicted_ratings if movieId not in rated_movies.values]
    
    # Return the top n recommendations
    return recommendations[:n_recommendations]


In [ ]:
# Get movie recommendations for a user
userId = 1
n_recommendations = 5
recommended_movies = recommend_movies(userId, n_recommendations)
print(f"Recommended movies for user {userId}: {recommended_movies}")

Recommended movies for user 1: [318, 720, 914, 904, 1283]
